In [1]:
import csv
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import umap.umap_ as umap
from scipy.linalg import eigh as largest_eigh
import sklearn

from scipy.linalg import orthogonal_procrustes

In [2]:
HiC_Exc = pd.read_csv("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/SAI_results/single_cell_eigens/Exc/eigen.txt", sep='\t', names=["chr", "start", "end", "compartment", "eigen"], index_col=False)
bin_df = pd.read_csv("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/gap.blacklist.hg38.1000000.4col.autosomal.bed", sep='\t', names=["chr","start","end","name"])
delete_bins = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Cross-Sample/1000K_resolution/delete_bins.npy")
bin_df_new = bin_df.drop(delete_bins).reset_index(drop=True)

In [43]:
bin_df_new[bin_df_new.name=="chr2_16"]

,chr,start,end,name
228,chr2,15000000,16000000,chr2_16


In [38]:
bin_df_new[bin_df_new.name=="chr2_200"]

,chr,start,end,name
384,chr2,199000000,200000000,chr2_200


In [39]:
bin_df_new[bin_df_new.name=="chr10_18"]

,chr,start,end,name
1387,chr10,17000000,18000000,chr10_18


In [4]:
bulk_feature = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Bulk-ATAC-seq/hfc/feature.npy")
CON_Exc_feature = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Cross-Sample/1000K_resolution/3_CON/cell_type/Exc/raw/feature.npy")
CON_Mic_feature = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Cross-Sample/1000K_resolution/3_CON/cell_type/Mic/raw/feature.npy")

In [5]:
bulk_emb = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Bulk-ATAC-seq/hfc/Embedding/corr_0_1_VERSE_32.npy")
CON_Exc_emb = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Cross-Sample/1000K_resolution/3_CON/cell_type/Exc/200_metacell/Embedding/corr_0_1_VERSE_32.npy")
CON_Mic_emb = np.load("/lv_scratch/scratch/share/scENCORE_brain/ZD_SS_ShareData/Cross-Sample/1000K_resolution/3_CON/cell_type/Mic/200_metacell/Embedding/corr_0_1_VERSE_32.npy")

In [9]:
CON_Exc_emb = (CON_Exc_emb-np.min(CON_Exc_emb))/(np.max(CON_Exc_emb)-np.min(CON_Exc_emb))
CON_Mic_emb = (CON_Mic_emb-np.min(CON_Mic_emb))/(np.max(CON_Mic_emb)-np.min(CON_Mic_emb))

In [10]:
R, _ = orthogonal_procrustes(CON_Exc_emb, CON_Mic_emb)
transfer_Exc = CON_Exc_emb @ R

In [16]:
D = np.sqrt(np.sum((transfer_Exc - CON_Mic_emb)**2, axis=1))

In [20]:
D

array([0.6319639 , 0.5313983 , 0.57954955, ..., 0.5485235 , 0.5908312 ,
       0.73517495], dtype=float32)

In [32]:
1-np.exp(-1)

0.6321205588285577

In [30]:
CSS = 1-np.exp(-D)

In [31]:
CSS[384]

0.536148

In [40]:
CSS[1387]

0.5861143

In [25]:
np.argsort(D)

array([2097, 1558, 2321, ..., 1389, 1480,  751])

In [23]:
D[2097], D[751]

(0.32959774, 0.929872)

In [26]:
np.where(np.argsort(D) == 384)

(array([2074]),)

In [28]:
2329-2074

255

In [41]:
np.where(np.argsort(D) == 1387)

(array([2320]),)

In [42]:
2329-2320

9